<a href="https://colab.research.google.com/github/wurDevTim/Workhshop_P4P/blob/main/introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welkom to the P4P workshop!
In this notebook, we will show how to analyse cropreporter data with plantcv.
The first step is downloading the data:
https://drive.google.com/file/d/1qaFPIgGrOB_t1xNH8wub1LzzmV8uvSZG/view?usp=sharing

On the left click the folder icon and upload the zip file

In [23]:
!unzip /npec_tomato.zip -d /npec_tomato



Archive:  /npec_tomato.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /npec_tomato.zip or
        /npec_tomato.zip.zip, and cannot find /npec_tomato.zip.ZIP, period.


Install Plantcv

In [5]:
!pip install git+https://github.com/wurDevTim/plantcv

  Cloning https://github.com/wurDevTim/plantcv to /tmp/pip-req-build-pty7g0ts
  Running command git clone --filter=blob:none --quiet https://github.com/wurDevTim/plantcv /tmp/pip-req-build-pty7g0ts
  Resolved https://github.com/wurDevTim/plantcv to commit 5fe40c7f6a11082663ab1201f573da891cc67362
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 52.7 MB/s eta 0:00:00
  Created wheel for plantcv: filename=plantcv-3.14.2+2426.g5fe40c7f-py3-none-any.whl size=328176 sha256=b22f7064d3300880b2aa33b3f7d833a70ca82aa84aca33e978892dc302e4d018
  Stored in directory: /tmp/pip-ephem-wheel-cache-wd4dwz6s/wheels/7e/d4/bc/5ebe3a5b6c1a61c110d2b127f8055e0b00503599bb5e487a47
Successfully built plantcv


In [ ]:
NPEC_tomato_drive = "https://drive.google.com/drive/folders/1F38XYnINWWB5k169j0zjj_UpaKXz6y7z?usp=sharing"

for hdr_file in glob(NPEC_tomato_drive):
